A notebook to calculate comparison metrics for the GAN reconstructions, such as the Pearson correlation coefficient and RMSE.

In [1]:
# Imports list
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate


In [2]:
# Load the samples (there are 20 sets of data):

data_dir = "/share/gpu0/jjwhit/samples/epoch-277/"

np_gts = {f"{i}": np.load(data_dir+f"np_gt_{i}.npy") for i in range(1, 11)}
np_samps = {f"{i}": np.load(data_dir+f"np_samps_{i}.npy") for i in range(1, 11)}
np_avgs = {f"{i}": np.load(data_dir+f"np_avgs_{i}.npy") for i in range(1, 11)}
np_stds = {f"{i}": np.load(data_dir+f"np_stds_{i}.npy") for i in range(1, 11)}
np_kss = {f"{i}": np.load(data_dir+f"np_kss_{i}.npy") for i in range(1, 11)}

#array['i'] to access the i-th ground truth, reconstruction, standard deviation, and Kaiser-Squires map; 
# for individual posterior samples np_samps['i']['j'] where j is in range [0,31].

# mask =  np.load(
#     data_dir + 'cosmos_mask.npy', allow_pickle=True
# ).astype(bool)
mask =  np.load('/home/jjwhit/rcGAN/mass_map_utils/cosmos/cosmos_mask.npy', allow_pickle=True
).astype(bool)


In [3]:
def rmse(a:np.ndarray, b:np.ndarray, mask:bool)->float:
    '''
    args:
        a (np.ndarray): ground truth
        b (np.ndarray): reconstruction
        mask (bool): mask
    returns:
        rmse (float): root mean squared error
    '''
    a = a[mask==1]
    b = b[mask==1]
    return(np.sqrt(np.mean(np.square(a-b))))

def pearsoncoeff(a:np.ndarray, b:np.ndarray, mask:bool)->float:
    '''
    args:
        a (np.ndarray): ground truth
        b (np.ndarray): reconstruction
        mask (bool): mask
    returns:
        pearson (float): Pearson correlation coefficient
    '''
    a = a[mask==1]
    b = b[mask==1]
    a -= np.mean(a)
    b -= np.mean(b)
    num = np.sum(a*b)
    denom = np.sqrt(np.sum(a**2)*np.sum(b**2))
    return num/denom

def psnr(a:np.ndarray, b:np.ndarray, mask:bool)->float:
    '''
    args:
        a (np.ndarray): ground truth
        b (np.ndarray): reconstruction
        mask (bool): mask
    returns:
        psnr (float): peak signal-to-noise ratio
    '''
    a = a[mask==1]
    b = b[mask==1]
    mse = np.mean((a-b)**2)
    r = a.max()
    return 10*np.log10(r/mse)

def SNR(a:np.ndarray, b:np.ndarray, mask:bool)->float:
    '''
    args:
        a (np.ndarray): ground truth
        b (np.ndarray): reconstruction
        mask (bool): mask
    returns:
        snr (float): signal-to-noise ratio
    '''
    a = a[mask==1]
    b = b[mask==1]
    signal = np.mean(a**2)
    noise = np.mean((a-b)**2)
    return 10*np.log10(signal/noise)

In [4]:
# Calculate the RMSE and Pearson correlation coefficient for the Kaiser-Squires and GAN reconstructions for the 20 samples

r_ks = []
r_gan = []
rmse_ks = []
rmse_gan = []
psnr_ks = []
psnr_gan = []
snr_ks = []
snr_gan = []

for n in range(1,11):
    num = f'{n}'
    # Gets the real component of the truth, Kaiser-Squires, and GAN reconstruction
    gt = np_gts[num].real
    ks = np_kss[num].real
    gan = np_avgs[num].real

    r_gan.append(pearsoncoeff(gt, gan, mask))
    r_ks.append(pearsoncoeff(gt, ks, mask))

    rmse_ks.append(rmse(ks, gt, mask))
    rmse_gan.append(rmse(gan, gt, mask))

    psnr_ks.append(psnr(gt, ks, mask))
    psnr_gan.append(psnr(gt, gan, mask))

    snr_ks.append(SNR(gt, ks, mask))
    snr_gan.append(SNR(gt, gan, mask))


In [5]:
# Size of each list should be 20 elements
print(len(r_ks), len(r_gan), len(rmse_ks), len(rmse_gan), len(psnr_ks), len(psnr_gan), len(snr_ks), len(snr_gan))

10 10 10 10 10 10 10 10


In [6]:
head = ['Metric', 'Kaiser-Squires', 'GAN']
table = [['Pearson (high)', np.mean(r_ks), np.mean(r_gan)],['RMSE (low)', np.mean(rmse_ks), np.mean(rmse_gan)],['PSNR (high)', np.mean(psnr_ks), np.mean(psnr_gan)],['SNR (high)', np.mean(snr_ks), np.mean(snr_gan)]]

print(tabulate(table, headers=head, tablefmt='pretty'))

+----------------+----------------------+----------------------+
|     Metric     |    Kaiser-Squires    |         GAN          |
+----------------+----------------------+----------------------+
| Pearson (high) |  0.6141927748024927  |  0.5698968768119812  |
|   RMSE (low)   | 0.022052101223097267 | 0.022196171805262566 |
|  PSNR (high)   |  30.520906919697886  |  30.47011923789978   |
|   SNR (high)   |  2.0643855884724274  |  2.0135959088802338  |
+----------------+----------------------+----------------------+


Making the Kaiser-Squires maps:
(You don't need to run this, but here is how the Kaiser-Squires maps were made)

In [4]:
import sys
dir = '/home/jjwhit/rcGAN/'
sys.path.append(dir)
from data.lightning.MassMappingDataModule import MMDataTransform
from mass_map_utils.scripts.ks_utils import backward_model
from scipy import ndimage

/home/jjwhit/rcGAN/fastmri/__init__.py:16: UserWarning: Could not retrieve fastmri version!
  warnings.warn("Could not retrieve fastmri version!")


In [5]:
# std1 = np.load(data_dir + 'cosmos_std1.npy', allow_pickle=True)
# std2 = np.load(data_dir + 'cosmos_std2.npy', allow_pickle=True)

std1 = np.load(dir + 'mass_map_utils/cosmos/' + 'cosmos_std1.npy', allow_pickle=True)
std2 = np.load(dir + 'mass_map_utils/cosmos/' + 'cosmos_std2.npy', allow_pickle=True)

In [6]:
kernel = MMDataTransform.compute_fourier_kernel(300)
np_kss = {}
for i in range(1, 11):
    num = f'{i}'
    gamma_sim = MMDataTransform.forward_model(np_gts[num], kernel) + (
                std1 * np.random.randn(300, 300) + 1.j * std2 * np.random.randn(300,300)
            )
    gamma_sim *= mask
    backward = backward_model(gamma_sim, kernel)
    ks = ndimage.gaussian_filter(backward, sigma=1/.29)

    np.save(data_dir+f'np_kss_{i}.npy', ks)
    # np_kss[num] = ks

/home/jjwhit/rcGAN/mass_map_utils/scripts/ks_utils.py:16: RuntimeWarning: divide by zero encountered in divide
  𝓕𝜅 = 𝓕γ / 𝒟  # Map convergence onto shear


In [ ]:
# TODO: Add the graph showing metrics as a function of n-image average

# n_values = range(1, 33)
# plt.plot(n_values, r_n, marker='.')
# plt.xlabel('Number of Samples Averaged, N')
# plt.ylabel('Pearson Correlation Coefficient, r')
# plt.title('Pearson correlation coefficient vs Number of Samples used in Reconstruction')
# plt.grid(True)
# plt.show()


# psnr_values_per_map = [[] for _ in range(10)]

# for i in range(10):
#     map_name = f'{i+1}'
#     psnr_vals = []
#     for p in range(1, 33):
#         avg_img = np.mean(np_samps[map_name][:p].real, axis=0)
#         psnr_instance = psnr(np_gts[map_name].real, avg_img) 
#         psnr_vals.append(psnr_instance)
#     psnr_values_per_map[i] = psnr_vals

# mean_psnr = np.mean(psnr_values_per_map, axis=0)
# std_dev_psnr = np.std(psnr_values_per_map, axis=0)


# n_values = range(1, 33)
# plt.errorbar(n_values, mean_psnr, yerr=std_dev_psnr, fmt='o', linestyle='-')
# plt.xlabel('Number of Samples Averaged, N')
# plt.ylabel('PSNR')
# plt.title('Average PSNR')
# plt.grid(True)
# plt.show()